In [1]:
print('ok')

ok


In [2]:
%pwd

'e:\\NLP_learning\\Projects\\medical_chat_bot\\medical_chat_bot\\research'

### we can change the direcotry with the help of this

In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'e:\\NLP_learning\\Projects\\medical_chat_bot\\medical_chat_bot'

In [20]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [2]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob='*.pdf',
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [5]:
extracted_docs = load_pdf_file(data='data/')

In [8]:
# extracted_docs

In [6]:
def text_spltter(extracted_docs):
    text_split = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    text_chunks = text_split.split_documents(extracted_docs)
    return text_chunks
    

In [7]:
text_chunks = text_spltter(extracted_docs)


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
def embedding_creation():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [10]:
embeddings = embedding_creation()

C:\Users\ADESH\AppData\Local\Temp\ipykernel_17468\1981112128.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\ADESH\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
t = embeddings.embed_query('hello world')

c:\Users\ADESH\anaconda3\envs\medibot\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## Creating the vector database

In [14]:
pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
    --------------------------------------- 0.3/14.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.9 MB 762.0 kB/s eta 0:00:19
   - -------------------------------------- 0.5/14.9 MB 762.0 kB/s eta 0:00:19
   -- ------------------------------------- 0.8/14.9 MB 729.2 kB/s eta 0:00:20
   -- ------------------------------------- 0.8/14.9 MB 729.2 kB/s eta 0:00:20
   -- ------------------------------------- 1.0/14.9 MB 718.9 kB/s eta 0:00:20
   --- ------------------------------------ 1.3/14.9 MB 737.4 kB/s eta 0:00:19
   ---- ----------------------------------- 1.6/14.9 MB 838.6 kB/s eta 0:00:16
   ---- ----------------------------------- 1.8/14.9 MB 898.8 kB/s eta 0:00:15
   ----- ---------------------------------- 2.1/14.9 MB 910.2 kB/s eta 0:00:15
   ------

In [15]:
vector_store = FAISS.from_documents(text_chunks, embeddings)

c:\Users\ADESH\anaconda3\envs\medibot\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### Retriever

In [16]:
retriver = vector_store.as_retriever(search_kwargs={'k': 4})

In [18]:
retriver.invoke('symptoms of headache')

c:\Users\ADESH\anaconda3\envs\medibot\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='9f1cd1a8-de57-4d79-99b9-b4af63faa03d', metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-08-01T10:48:16+00:00', 'source': 'data\\The_Merck_Manual_of_Diagnosis_and_Therapy_2011_-_19th_Edn.......-53-874.pdf', 'total_pages': 822, 'page': 602, 'page_label': '603'}, page_content='Family history should note any family history of migraine headaches.\nPhysical examination:\n Vital signs, including temperature, are measured.\nIf the diagnosis of a transient ischemic attack is under consideration, a complete neurologic examination\nis done. The facial skin is inspected for vesicles or ulcers in the V\n1\n distribution (ophthalmic division of the\ntrigeminal nerve), and the temples are palpated for pulses, tenderness, or nodularity over the course of\nThe Merck Manual of Diagnosis & Therapy, 19th Edition Chapter 60. Approach to the Ophthalmologic Patient\n645'),
 Document(id='265c8ed1-2332-440e-99ec-a43b14c29c8f', metadata={'producer': 'iLove

### Augmentation

In [21]:
prompt = PromptTemplate(
    template="""
    You are a helpful assistant.
    Answer ONLY from the provided transcript context.
    If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables=['context','question']
)

In [33]:
question = 'what are the treatments on Hiccups?'
retrived_docs = retriver.invoke(question)

c:\Users\ADESH\anaconda3\envs\medibot\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [34]:
context = "\n\n".join(doc.page_content for doc in retrived_docs)
context

'PaCO\n2\n can be increased and diaphragmatic activity can be inhibited by a series of deep breath-holds or\nby breathing deeply in to and out of a paper bag. (CAUTION: \nPlastic bags can cling to the nostrils and\nshould not be used\n.) Vagal stimulation by pharyngeal irritation (eg, swallowing dry bread, granulated\nsugar, or crushed ice; applying traction on the tongue; stimulating gagging) may work. Numerous other\nfolk remedies exist.\nPersistent hiccups are often recalcitrant to treatment. Many drugs have been used in anecdotal series.\nBaclofen, a \nγ\n-aminobutyric acid agonist (5 mg po q 6 h increasing to 20 mg/dose), may be effective. Other\ndrugs include chlorpromazine 10 to 50 mg po tid as needed, metoclopramide 10 mg po bid to qid, and\nvarious anticonvulsants (eg, gabapentin). Additionally, an empiric trial of proton pump inhibitors may be\ngiven. For severe symptoms, chlorpromazine 25 to 50 mg IM or IV can be given. In intractable cases, the\n\nTable 7-4\n) are both nume

In [35]:
final_propmt = prompt.invoke({'context':context, 'question':question})

### Generation

In [36]:
from langchain_community.chat_models import ChatOllama

# This line uses your free, local Phi-3 model via Ollama
llm = ChatOllama(model="phi3:mini", temperature=0.2)

# The rest of your code works exactly the same!
response = llm.invoke(final_propmt)
print(response)

content="Treatment for persistent and intractable hiccups includes identifying any underlying problems, which may require specific treatment. If no cause is found or if symptoms persist despite this approach, various simple measures can be tried: increasing PaCO2 by holding one's breath deeply in a paper bag (not using plastic bags), and trying to increase diaphragmatic activity through deep breath-holds are mentioned as methods for relief. Additionally, drugs such as Baclofen, chlorpromazine, metoclopramide, various anticonvulsants like gabapentin, proton pump inhibitors may be used empirically based on the patient's history and response to previous treatments. In severe cases or when hiccups are intractable after 1 month of symptoms without an identifiable cause, chlorpromazine can also be administered intramuscularly (IM) or intravenously (IV)." additional_kwargs={} response_metadata={'model': 'phi3:mini', 'created_at': '2025-08-03T11:58:50.718927Z', 'message': {'role': 'assistant',